## Activate GPU

1.   Runtime menu
2.   Change runtime type
3.   Hardware accelerator -> GPU

## Get Foolbox

#### Solution

In [ ]:
!pip3 install foolbox==3.0.0b1
# !pip3 install git+https://github.com/bethgelab/foolbox.git

In [ ]:
import foolbox as fb

## Get a model

Get a pertrained PyTorch or TensorFlow model, e.g. `torchvision.models.resnet18` or `tf.keras.applications.ResNet50`.

#### PyTorch Solution

In [ ]:
import torch
import torchvision

In [ ]:
torch.__version__

In [ ]:
torch.cuda.is_available()

In [ ]:
model = torchvision.models.resnet18(pretrained=True)

In [ ]:
model = model.eval()

## Turn your PyTorch model into a Foolbox model

Don't forget to specify the correct bounds and preprocessing!

#### PyTorch Solution

In [ ]:
# PyTorch ResNet18
preprocessing = dict(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], axis=-3)
bounds = (0, 1)
fmodel = fb.PyTorchModel(model, bounds=bounds, preprocessing=preprocessing)

## Transform bounds

In the following, we want to work with a model that has `(0, 1)` bounds. Use `fmodel.transform_bounds`.

#### Solution

In [ ]:
fmodel = fmodel.transform_bounds((0, 1))

In [ ]:
assert fmodel.bounds == (0, 1)

## Get some test images

Get a batch of 16 images and the corrresponding labels. You can use `foolbox.utils.samples` to get up to 20 images, but you can also you your own data loader.

#### Solution

In [ ]:
images, labels = fb.utils.samples(fmodel, dataset='imagenet', batchsize=16)

## Check the accuracy of your model to make sure you specified the correct preprocessing

#### Solution

In [ ]:
fb.utils.accuracy(fmodel, images, labels)

In [ ]:
type(images), images.shape

In [ ]:
type(labels), labels.shape

## Run LinfDeepFool

#### Solution

In [ ]:
attack = fb.attacks.LinfDeepFoolAttack()

In [ ]:
raw, clipped, is_adv = attack(fmodel, images, labels, epsilons=0.03)

In [ ]:
is_adv

## Use EagerPy tensors and rerun the attack

#### Solution

In [ ]:
import eagerpy as ep

In [ ]:
images = ep.astensor(images)
labels = ep.astensor(labels)

In [ ]:
raw, clipped, is_adv = attack(fmodel, images, labels, epsilons=0.03)

In [ ]:
is_adv

In [ ]:
is_adv.float32().mean().item()

## Using the Misclassification criterion explicitly

#### Solution

In [ ]:
criterion = fb.criteria.Misclassification(labels)

In [ ]:
raw, clipped, is_adv = attack(fmodel, images, criterion, epsilons=0.03)

In [ ]:
is_adv

## Run the attack using many epsilons

#### Solution

In [ ]:
import numpy as np

In [ ]:
epsilons = np.linspace(0.0, 0.005, num=20)

In [ ]:
raw, clipped, is_adv = attack(fmodel, images, labels, epsilons=epsilons)

In [ ]:
is_adv.shape

In [ ]:
is_adv.float32().mean(axis=-1)

In [ ]:
robust_accuracy = 1 - is_adv.float32().mean(axis=-1)

In [ ]:
robust_accuracy

## Plot the robust accuracy as a function of epsilon

#### Solution

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(epsilons, robust_accuracy.numpy())

## Run a targeted attack

#### Solution

In [ ]:
labels

In [ ]:
target_classes = (labels + 200) % 1000

In [ ]:
target_classes

In [ ]:
criterion = fb.criteria.TargetedMisclassification(target_classes)

In [ ]:
attack = fb.attacks.L2CarliniWagnerAttack(steps=100)
# Note: 100 is too little -> results will be bad = perturbations will be relatively large (but 1000 takes much longer)

In [ ]:
# epsilons = np.linspace(0.0, 10.0, num=20)
epsilons = None

In [ ]:
advs, _, is_adv = attack(fmodel, images, criterion, epsilons=epsilons)

In [ ]:
is_adv

In [ ]:
fb.distances.l2(images, raw)

In [ ]:
# attack_success_rate = is_adv.float32().mean(axis=-1)

In [ ]:
# plt.plot(epsilons, attack_success_rate.numpy())

## Visualizing adversarial examples and perturbations

#### Solution

In [ ]:
fb.plot.images(images)

In [ ]:
fb.plot.images(advs)

In [ ]:
fb.plot.images(advs - images, n=4, bounds=(-1, 1), scale=4.)

## Continuing from here ...



*   Repeating an attack (`attack = attack.repeat(3)`)
*   Getting the per-sample worst-case over multiple attacks
    * stack attack results and take max over the attacks before taking the mean over samples
*   Gradient estimators (`fb.gradient_estimators.*`)
*   Transfer attacks using gradient substitution (see examples)

